In [4]:
!nvidia-smi

Fri Dec  4 15:19:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sys

In [10]:



#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--labels', type=int, required=True, help='model output')
#args = parser.parse_args()

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True,
    cache_dir='.', cache_subdir='data')
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

input_width = 6
LABEL_OPTIONS = 2


In [18]:
x = np.array([1,2,3,4,5,6])
print(x[-1])
print(x[:-3])
print(x[-3:])

6
[1 2 3]
[4 5 6]


In [19]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        input_indeces = np.arange(self.input_width)
        inputs = features[:, :-6, :]
            
        labels = features[:, -6:, :]
        num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        # vedere se funge
        labels.set_shape([None, self.input_width, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                # Targets None because we have the targets incorporated in the dataset
                targets=None,
                sequence_length=input_width+6,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


In [20]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [27]:
for element in train_ds:
    print(len(element))
    print(element[0].shape, element[1].shape)
    #print(element[0])
    sys.exit()

2
(32, 6, 2) (32, 6, 2)


SystemExit: 

## Models

In [31]:
MODEL_OPTIONS = "MLP"

units=2


if MODEL_OPTIONS == "MLP":
    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6, 2)),
    keras.layers.Dense(128, activation='relu', name='first_dense'),
    keras.layers.Dense(128, activation='relu', name='second_dense'),
    keras.layers.Dense(units, name='third_dense')
    ])


In [41]:
class MsMoMAE(tf.keras.metrics.Metric):
    def __init__(self, name='MsMoMAE', **kwargs):
        super(MsMoMAE, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros', shape=(2, ))
        self.count = self.add_weight(name='count', initializer='zeros')

    def reset_states(self):
        self.count.assign(tf.zeros_like(self.count))
        self.total.assign(tf.zeros_like(self.total))

        return

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.abs(y_pred - y_true)
        error = tf.reduce_mean(error, axis=1)
        error = tf.reduce_mean(error, axis=0)
        self.total.assign_add(error)
        self.count.assign_add(1)

        return

    def result(self):
        result = tf.math.divide_no_nan(self.total, self.count)

        return result


In [42]:
model.compile(optimizer='adam',
            loss=[tf.keras.losses.MeanSquaredError()],
            metrics=[MsMoMAE()])

print("Fit model on training data")

history = model.fit(
    train_ds,
    batch_size=32,
    epochs=1,
    validation_data=(val_ds)
)

print("Evaluate on test data")
loss, error = model.evaluate(test_ds, verbose=2)
print(error)

model.summary()

Fit model on training data


InvalidArgumentError:  Incompatible shapes: [32,6,2] vs. [32,2]
	 [[node mean_squared_error/SquaredDifference (defined at <ipython-input-42-5f0b170a7a22>:11) ]] [Op:__inference_train_function_3855]

Function call stack:
train_function
